# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:

# Output File (CSV)
output_data_file = "../output_data/cities.csv"
output_data_file_df = pd.read_csv(output_data_file)


output_data_file_df = output_data_file_df.dropna()
output_data_file_df.head()


,Unnamed: 0,City,Lat,lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Cape Town,-33.9258,18.4232,61.00,88,0,8.05,ZA,1619557323
1,1,Mogadishu,2.0371,45.3438,82.87,75,39,10.92,SO,1619557536
2,2,Qaanaaq,77.4840,-69.3632,38.46,93,100,5.23,GL,1619557536
3,3,San Juan,-31.5375,-68.5364,79.56,24,0,4.56,AR,1619557380
4,4,Meiktila,20.8667,95.8667,78.78,64,30,9.48,MM,1619557537


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 

# HINT: be sure to handle NaN values
output_data_file_df = output_data_file_df.dropna()

locations = output_data_file_df[["Lat", "lng"]].astype(float)
Humidity = output_data_file_df["Humidity"].astype(float)


# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
NarrowDownData_df = output_data_file_df.loc[(output_data_file_df['Max temp'] > 70) & (output_data_file_df['Max temp'] < 80) & (output_data_file_df['Wind Speed'] < 10) & (output_data_file_df['Cloudiness'] == 0)]
NarrowDownData_df = NarrowDownData_df.dropna()
NarrowDownData_df.head()

,Unnamed: 0,City,Lat,lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,San Juan,-31.5375,-68.5364,79.56,24,0,4.56,AR,1619557380
10,10,Saint-Philippe,-21.3585,55.7679,75.20,88,0,3.44,RE,1619557539
47,47,Saint-Joseph,-21.3667,55.6167,75.20,88,0,3.44,RE,1619557549
48,48,Phulbāni,20.4667,84.2333,77.34,45,0,3.62,IN,1619557549
95,95,Ribas do Rio Pardo,-20.4431,-53.7592,78.78,31,0,7.56,BR,1619557563


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = NarrowDownData_df
del hotel_df["Unnamed: 0"]
hotel_df["Hotel name"] = ''
hotel_df.head()

,City,Lat,lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel name
3,San Juan,-31.5375,-68.5364,79.56,24,0,4.56,AR,1619557380,
10,Saint-Philippe,-21.3585,55.7679,75.20,88,0,3.44,RE,1619557539,
47,Saint-Joseph,-21.3667,55.6167,75.20,88,0,3.44,RE,1619557549,
48,Phulbāni,20.4667,84.2333,77.34,45,0,3.62,IN,1619557549,
95,Ribas do Rio Pardo,-20.4431,-53.7592,78.78,31,0,7.56,BR,1619557563,


In [17]:
HotelName = []

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  

# Loop through narrowed down dataframe to get nearest hotel
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["lng"]

    LatLng = f"{lat},{lng}"

    params = {
        "location": LatLng, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    request = requests.get(base_url, params=params)
    response = request.json()

    HotelName.append(response["results"][0]["name"])
    
# Dataframe with nearest hotel
hotel_df["Hotel Name"] = HotelName
hotel_df.head()

,City,Lat,lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel name,Hotel Name
3,San Juan,-31.5375,-68.5364,79.56,24,0,4.56,AR,1619557380,,Hotel Provincial
10,Saint-Philippe,-21.3585,55.7679,75.20,88,0,3.44,RE,1619557539,,"Chambres d'hôte ""La Trinité"""
47,Saint-Joseph,-21.3667,55.6167,75.20,88,0,3.44,RE,1619557549,,"""Plantation Bed and Breakfast"
48,Phulbāni,20.4667,84.2333,77.34,45,0,3.62,IN,1619557549,,Hotel Raj Kamal
95,Ribas do Rio Pardo,-20.4431,-53.7592,78.78,31,0,7.56,BR,1619557563,,Pousada Avenida


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "lng"]]

In [19]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in HotelName])
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))